# Suivis Projet Info : Stellar Analytics
#### Axelle Refeyton . Jean Roelens

Le but de ce projet est initialement de permettre à un utilisateur de prendre une photo d'une étoile et de retrouver des informations dessus, notamment son nom, sa position ainsi que sa composition chimique qui peux être récupérée en fonction de son spectre lumineux.

Ressources utilisées : 
- [NIST Spectra Database](https://physics.nist.gov/PhysRefData/ASD/lines_form.html)
- [Documentation sur les spectres lumineux](https://atomic-spectra.net/)
- [Simbad, DB d'étoiles communautaire open source](https://simbad.u-strasbg.fr/simbad/)
- [MAST portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html)
- [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer)
- [Plate Solving](https://nighttime-imaging.eu/docs/master/site/advanced/platesolving)

En début de projet, beaucoup de temps fut consacré à la recherche de jeux de données et de documentation. Egalement, nous avons eu l'occasion de nous entretenir lors d'une conférence Zoom avec deux membres du CNES (Orphée Faucoz et Denis Standarovski) travaillant depuis plusieurs années sur un sujet similaire : L'identification d'exoplanètes en utilisant les spectres d'absorption.

Au cours de l'entretien ils nous ont clairement expliqué que ce que l'on souhaiter faire était une tâche difficile pour laquelle il n'y avait pas beaucoup de ressources, et que cela nécessiterait beaucoup de temps ne serait-ce que pour la compréhension de l'environnement dans lequel nous naviguons.

Nous avons parcouru beaucoup de bases de données (certaines gratuites et d'autres non) afin de trouver celles qui correspondrait au mieux à nos besoins. Au final nous avons retenu Simbad pour la récupération des données générales des étoiles et NIST pour récupérer le spectre d'émissions des élèments que peuvent composer une étoile, c'est-à-dire des gaz et des métaux. Malheureusement, aucune base ne nous permet de récupérer un grand nombre de spectres d'étoiles efficacement : il va donc falloir les générer.

Devant la complexité de la tâche, nous avons décidé de découper le projet en différentes partie et de ce concentrer pour le moment sur la partie spectre chimique. Cette tâche a été divisée en plusieurs étapes : la récupération des spectres des éléments, et la génération de spectres d'étoiles.

## Spectre chimique d'élément (J.Roelens)

L'objectif de cette partie est :
1. **Pour chaque éléments récupérer les différentes ionisations possible.**
2. **Extraire le tableau de spectres depuis NIST (en utilisant un scrapper)**
3. **Le formatter pour ne récupérer que les lignes fortes**
4. **Créer un dataset par élément et ionization avec les lignes fortes**

Pour commencer, il faut que je récupére les ionisations de chaque éléments. Etant une étape pas forcément très complexe,j'ai préféré le faire à la main en m'aidant de [atomic-spectra](https://atomic-spectra.net/spectrum.php) et des [Raies de Fraunhofer](https://fr.wikipedia.org/wiki/Raies_de_Fraunhofer). On en ressort cette liste : 
- Oxygène [I, II, III, IV, V, VI, VII, VIII]
- Hydrogènes [I]
- Sodium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI]
- Hélium [I, II]
- Mercure [I, II, III]
- Fer [I, II ,III , IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XVI]
- Magnésium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII]
- Calcium [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI]
- Titane [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII]
- Nickel [I, II, III, IV, V, VI, VII, VIII, IX, X, XI, XII, XIII, XIV, XV, XVI, XVII, XVIII, XIX, XX, XXI, XXII, XXIII, XXIV, XXV, XXVI, XXVII, XXVIII]

Maintenant que l'étape facile est faite, je vais pouvoir m'attaquer à l'étape la plus hardue : extraire les données depuis NIST. L'idée globale est de récupérer le HTML de la page NIST en donnant dans l'URL du site l'élément ainsi que son indice de ionisation.

En saisissant H I (pour Hydrogène Ionization 1) on obtiens cet URL :

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=H+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Plutôt long, mais ce qui est assez intéressant c'est que beaucoup de paramètres pourront être passés assez facilement par celle-ci. Comparons avec l'URL de O I pour voir où saisir l'élément et l'ionization 

https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra=O+I&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on

Comme on peut le voir l'élèment se trouve en début d'URL. Essayons cela.

In [1]:
import requests

START_URL = r"https://physics.nist.gov/cgi-bin/ASD/lines1.pl?spectra="
END_URL = r"&output_type=0&low_w=&upp_w=&unit=1&submit=Retrieve+Data&de=0&plot_out=0&I_scale_type=1&format=0&line_out=0&en_unit=0&output=0&bibrefs=1&page_size=15&show_obs_wl=1&show_calc_wl=1&unc_out=1&order_out=0&max_low_enrg=&show_av=2&max_upp_enrg=&tsb_value=0&min_str=&A_out=0&intens_out=on&max_str=&allowed_out=1&forbid_out=1&min_accur=&min_intens=&conf_out=on&term_out=on&enrg_out=on&J_out=on"

# Essayons de voir si on peut obtenir un code 200, mais avant cela construisons notre URL de maniére naïve pour ce test

URL = START_URL + "Ni+X" + END_URL

resp = requests.get(URL)
if resp.status_code == 200:
    print(f"Succés de la requéte")
else :
    print(f"Echec de la requéte")

Succés de la requéte


Maintenant que notre approche naïve fonctionne vérifions que l'intégralité des élèments et ionization fonctionne.

In [2]:
ELEMENT_DICT = {
    "O" : ["I", "II", "III", "IV", "V", "VI","VII"],
    "H" : ["I"],
    "Na" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI"],
    "He" : ["I","II"],
    "Hg" : ["I","II","III"],
    "Fe" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Mg" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII"],
    "Ca" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI"],
    "Ti" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII"],
    "Ni" : ["I", "II", "III", "IV", "V", "VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX","XXI","XXII","XXIII","XXIV","XXV","XXVI","XXVII","XXVIII"]
}

# Je vérifie la longueur de chacune des clès pour être sur de ne pas avoir fait d'erreur 
def element_dict_verification():
    for key in ELEMENT_DICT.keys():
        print(f"Elément : {key} Len : {len(ELEMENT_DICT[key])}")

# Et ensuite on test les requêtes

def get_200(r : requests.models.Response):
    if r.status_code != 200:
        return False
    return True
def multi_request_verification(d : dict):
    for k in d.keys():
        for v in range(len(d[k])):
            url = START_URL + k + "+" + d[k][v] + END_URL
            if not get_200(requests.get(url)):
                print(f"Error with el : {k} and ionization : {d[k][v]}")
            else : 
                print(f"Success with el : {k} and ionization : {d[k][v]}")
# multi_request_verification(ELEMENT_DICT)

Ok, on dirait bien qu'on a que des codes 200 pour l'ensemble des requêtes ! Malheureusement, on se rendra compte plus tard que NIST renvoie des code 200 même en cas de mauvais URL. Le seul moyen de savoir si la requête a bien abouti est de récupérer le tableau **"\<table>"** qui va contenir les données qui nous intéresse. 

L'idée est de voir si on peut récupérer la balise "table" contenant le styling que nous cherchons pour chaque lien. On s'aide de la librairie que nous allons le plus utiliser durant cette partie : "BeautifulSoup"

In [3]:
from bs4 import BeautifulSoup
from pprint import pprint

def url_builder(d : dict):
    resp = {}
    for key in d.keys():
        for v in d[key]:
            resp[key+v] = START_URL + key + "+" + v + END_URL
    return resp

In [6]:
urls = url_builder(ELEMENT_DICT)
for url in urls:
    req = requests.get(urls[url])
    soup = BeautifulSoup(req.text, 'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) != 1:
        print(f'error with url : {url}')
    else :
        continue

error with url : NiVI
error with url : NiVIII


On voit donc désormais que deux lien sont *mort* : celui pour le Nickel 6 et le Nickel 8. Hormis ça, l'ensemble des autres liens marche. Pour la prochaine étape il nous faut récupérer toute les rows contenant une donnée dans la colonne *Observed Wavelength Vac (nm)* ainsi que *Rel. Int.* Ce sont les deux colonnes qui nous intéressent pour la suite.

In [ ]:
# Pour commencer passont la logique précédente en fonction 

def get_table(req : requests.models.Response):
    soup = BeautifulSoup(req.text,'html.parser')
    tables = soup.find_all('table')
    target_table = [table for table in tables if 'background-color:#FFFEEE;' in table.get('style', '')]
    if len(target_table) > 0 :
        return target_table[0]
    else :
        return None

# On va traiter seulement le premier question de rapidité depuis python 3.6 les dicts sont ordonnées du coup le premier élèment sera toujours OI
urls = url_builder(ELEMENT_DICT)
oxygen_one = list(urls.items())[0]
table = get_table(requests.get(oxygen_one[1]))
# Maintenant on va "drill down" la table afin d'obtenir tout les Tbody puis les Tr qui nous intéressent.
tbodies = table.find_all('tbody')
# Seul les tbody pair nous intéresse car ils contiennent la data en question
evenTBodies = tbodies[1::2]
# Ensuite pour chaque tbody on veut récupérer toute les balises tr
# firstTBody = evenTBodies[0]
# Holder pour les values récupéré afin d'en faire un dataframe par la suite
All_Observed_Wavelength = []
All_Relative_Intensities = []

# for tbody in evenTBodies :
#     for tr in tbody.find_all('tr'):
#         all_td = tr.find_all('td')
#         if len(all_td) >= 2:
#             Obs_Wav = all_td[0].text.strip()
#             Rel_Int = all_td[2].text.strip()
#             if Obs_Wav != "" and Rel_Int != "":
#                 All_Observed_Wavelength.append(Obs_Wav)
#                 All_Relative_Intensities.append(Rel_Int)
# pprint(All_Observed_Wavelength)

# Tiens on remarque que la structure du HTML doit avoir un petite erreur car lors du print de All_Observed_Wavelength 
# on remarque qu'on print les bonnes colonnes avec les bonnes valeur mais, on a 1 fois le tableau 1, 2 fois le tableau 2, et finalement 3 fois le tableau
# 3 ce qui nous indique que l'on peut parcourir une seul fois le tableau 1 et récupérer toutes les données.

tbody = tbodies[1]
for tr in tbody.find_all('tr'):
    all_td = tr.find_all('td')
    if len(all_td) >= 2:
        obs_wav = all_td[0].text.strip()
        rel_int = all_td[2].text.strip()
        if obs_wav != "" and rel_int != "":
            All_Observed_Wavelength.append(obs_wav)
            All_Relative_Intensities.append(rel_int)
import pandas as pd

df = pd.DataFrame({
    'wavelength' : All_Observed_Wavelength,
    'intensities': All_Relative_Intensities,
    'chemical' : 'O I'
})

print(df.describe())

       wavelength intensities chemical
count         174         174      174
unique        157          48        1
top       94.8686         120      O I
freq            3          18      174


## Génération de spectre d'étoile (A.Refeyton)

<font color='#FF5733'>Attention : cette partie comportera des blocs de code mais ceux-ci seront plus à titre indicatif que réellement fonctionnel étant donné le besoin de jeux de données importants pour que ceux-ci fonctionne.</font>

Pour la création de spectre, l'idée va être d'exploiter les données scrappées de manière à sélectionner parmis elles un nombre X d'éléments et de créer une sorte de copycat de spectres d'étoiles à partir de ces élèments.

Pour commencer, il faut récupérer la liste des éléments précédemment scrappés.

In [15]:
import os
import pandas as pd

def get_elements_list(path):
    """
    path : chemin vers le dossier contenant tout les csv éléments
    """
    filenames = os.listdir(path)
    dataframes = []
    for n in filenames:
        data = pd.read_csv(path + "\\" + n)
        # On nomme le dataframe en fonction de l'élément
        data.Name = n[:-4]
        dataframes.append(data)
    return dataframes

Une fois les dataframes nommés et stockés, on va pouvoir les exploiter et générer nos spectres. On récupère d'abord les données d'un certains nombre de dataframe élément qu'on injecte ensuite dans un dataframe vide puis qu'on remplit de bruit. En parallèle, on créée un autre dataframe qui va nous permettre de stocker les labels correspondants à chaque spectre. Chaque élément possible est représenté par une colonne. On note la présence d'un élément par un 1 et son absence par un 0.

In [16]:
import numpy as np
import random as rand

# hyperparamètres
min_elems_used = 4
max_elems_used = 5
min_relint_noise = 0
max_relint_noise = 30
min_wl_noise = 0
max_wl_noise = 1100
min_n_noise = 1500
max_n_noise = 2000

def generate_spectrums(base_elements, n):
    """
    base_elements : Ensemble des dataframes éléments
    n : nombre de dataframe à générer
    """
    max_elems = len(base_elements)
    list_df = []
    matrice_labels = pd.DataFrame(columns=[x.Name for x in base_elements]) 

    # On vérifie qu'il y ait assez d'éléments
    if max_elems < min_elems_used:
        raise ValueError(f'Not enough data, {max_elems} elements present')
    
    for i in range(n):
        # création du df resultat en y ajoutant un nombre aléatoire d'éléments de base compris entre 2 et 5
        rand_elems = rand.choices(base_elements, k=rand.randrange(min_elems_used, max_elems if max_elems < max_elems_used else max_elems_used))
        # initialisation des colonnes
        matrice_labels.loc[i] = [0] * max_elems

        # Changement du 0 en 1 pour les éléments présents
        for elem in rand_elems:
            matrice_labels.at[i, elem.Name] = 1
            
        df_res = pd.concat(rand_elems, ignore_index=True)

        # Génération du bruit
        n_noise = rand.randint(1500, 2000)
        # Génération des wavelengths pour chacun des bruits
        rand_wls = [rand.uniform(0,1100) for x in range(n_noise)]

        # Ajout du bruit au dataset existant
        for n in range(0,n_noise):
            df_res.loc[len(df_res)] = {"wavelength": round(rand_wls[n], 1), "relint": round(rand.uniform(0, 50), 1)} 
        
        list_df.append(df_res.sort_values("wavelength"))
        
    return list_df, matrice_labels

On récupère ainsi une liste de dataframe représentant chacun un spectre ainsi qu'un dataframe contenant les labels correspondants.
Il est maintenant temps d'exploiter ces données.

## Exploitation des spectres construis

Attaquons-nous à la partie qui nous intéresse : exploiter les spectres afin de déterminer s'il est possible de trouver les différents éléments qui le composent

Pour ce faire, nous allons devoir utiliser un type de modèle un peu particulier : la classification supervisée multi-label. Il s'agit d'un type de modèle capable de classer une observation en lui affectant un à plusieurs labels différents, à savoir nos éléments.

Mais avant d'intégrer le modèle, il faut façonner nos données en un format que notre modèle peut digérer.

In [8]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings # Permet d'ignorer certains warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

La première étape va être d'"aplatir" nos dataframes. On récupère chaque cellule contenant de la donnée et on l'ajoute à une liste, qu'on ajoute ensuite à un dataframe contenant tout les résultats. De cette manière, une ligne de ce dataframe contiendra toutes les données relatives à un spectre.

In [9]:
def shape(list_df):
    result_df = pd.DataFrame()

    for df in list_df:
        # Aplatir le df en liste de x valeurs
        data_flat = df.to_numpy().flatten()

        # ajouter des colonnes au dataframe s'il n'y en a pas assez
        while len(data_flat) > len(result_df.columns):
            result_df[len(result_df.columns)] = 0

        # Si le df aplatis à moins de colonnes que le df de résultat on rajoute des colonnes au df aplatis et on y attribue la valeur nan (0)
        if len(data_flat) < len(result_df.columns):
            data_flat = np.concatenate([data_flat, [0] * (len(result_df.columns) - len(data_flat))])
        
        # ajout de la ligne
        result_df.loc[len(result_df)] = data_flat

    return result_df

Le problème, c'est qu'on se retrouve avec un dataframe contenant parfois des dizaines de milliers de colonnes. Afin d'y remédier, on a recours a l'ACP (analyse de composants principaux). Cette technique permet de réduire drastiquement le nombre de colonnes sans perdre les relations entre les données.

In [10]:
def pca(df, nb_components):

    # Transforme le dataframe pour que les valeurs soit comprise en 0 et 1
    data_scaled = MinMaxScaler().fit(df).transform(df)

    # PCA 
    result_df = PCA(n_components=nb_components).fit_transform(data_scaled)

    return result_df

Pas très impressionant en effet ! Et pourtant c'est cette méthode qui va nous faire gagner un temps considérable.

Bien ! Maintenant que tout est prêt nous allons pouvoir intégrer le modèle.

<font color='#FF5733'>Les modèles ne tourneront pas sur le Jupyter Notebook pour des raisons de baisse de performance</font>

In [20]:
from sklearn.model_selection import train_test_split
import xgboost as xgb # Le modèle de multi labelling
from sklearn.metrics import recall_score
from statistics import mean

# Pour commencer on récupére les élements
elements = get_elements_list("./generation_spectre/elements")

# Génére les spectres aléatoires + les labels associés
spectrums, labels = generate_spectrums(elements, 10)

# Transforme la liste de df en 1 df
data_shaped = shape(spectrums)

# On resize la données avec pca
data_scaled = pca(data_shaped, 5)

# On split la données pour finalement essayer notre modèle
X_train, X_val, y_train, y_val = train_test_split(data_scaled, labels.to_numpy(), test_size=0.8, random_state = 42)

model = xgb.XGBClassifier(tree_method="hist")
model.fit(X_train, y_train)
prediction = model.predict(X_val)

# Calculation de précision
overall_precision = ['%.2f' % elem for elem in recall_score(y_val, prediction, average=None)]
positive_precision = []
for i in range(len(y_val)):
    for j in range(len(y_val[i])):
        if y_val[i][j] == 1:
            positive_precision.append(prediction[i][j])
positive_precision = mean(positive_precision)

print(f"Score by label : {overall_precision}")
print(f"Positive labels precision : {positive_precision}")


Score by label : ['0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00', '0.00']
Positive labels precision : 0.0


C:\Users\littl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Sans surprise avec peu de données générées le score est nul (pour le coup vraiment nul car on est à 0) mais sur des jeux plus gros et avec plus de temps, on obtiens de 11% (avec 2000 spectres) à 13,8% (avec 7000 spectres).
Le résultat n'étant pas satisfaisant, on décide d'utiliser une deuxième méthode d'implémentation utilisant plusieurs modèles.

Le principe est de séparer les différents labels à trouver et de les répartir entre plusieurs modèles afin d'alléger la charge de travail.

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from statistics import mean
from sklearn.metrics import recall_score

max_labels_per_df = 5

# Les premières étapes sont similaires
# On récupére les éléments
elements = get_elements_list("./generation_spectre/elements")

# Génére les spectres aléatoires + les labels associés
spectrums, labels = generate_spectrums(elements, 10)

# Transforme la liste de df en 1 df
data_shaped = shape(spectrums)

# On resize la données avec pca
data_scaled = pca(data_shaped, 5)

# On slice ensuite les labels pour que chaque modèle s'occupe d'un morceau
list_sliced_labels = []

for i in range(len(labels.columns)//max_labels_per_df):
    list_sliced_labels.append(labels.iloc[:,i * max_labels_per_df : (i+1) * max_labels_per_df])

# Sans oublier les dernières colonnes
list_sliced_labels.append(labels.iloc[:, -(len(labels.columns) % max_labels_per_df) :])

# On instancie un modèle pour chaque groupe de label
prevision = pd.DataFrame()
expected = pd.DataFrame()

for df in list_sliced_labels:
    # Préparation des données
    X_train, X_val, y_train, y_val = train_test_split(data_scaled, df.to_numpy(), test_size=0.8, random_state=42)

    # Training
    model = XGBClassifier(tree_method="hist")
    model.fit(X_train, y_train)

    # Predicting
    result = model.predict(X_val)
    prevision = pd.concat([prevision, pd.DataFrame(result)], axis=1)
    expected = pd.concat([expected, pd.DataFrame(y_val)], axis=1)

# Finalement, on calcule la précision comme avant
prevision = prevision.to_numpy()
expected = expected.to_numpy()
map(float, overall_precision)
positive_precision = []
for i in range(len(expected)):
    for j in range(len(expected[i])):
        if expected[i][j] == 1:
            positive_precision.append(prediction[i][j])
positive_precision = mean(positive_precision)

print(f"positive labels precision : {positive_precision}")

positive labels precision : 0.0


Encore une fois un résultat nul avec 10 spectres, mais avec un jeux de données de 2000 puis 7000, on obtient respectivement 12% et 14%.

## Révision de la mise en forme

Le résultat est très décevant. Pour environ 10h de génération de donnée et un total de 7000 spectres, on n'atteint même pas les 20% de précision. En explorant la donnée on se rend compte qu le nombre de valeurs de longueur d'onde est beaucoup trop élevé (les longueurs d'ondes allant parfois jusqu'au 4ème chiffre après la virule), ce qui noie le modèle d'informations inutiles.

Nous décidons donc de changer notre approche.

Plutôt que de récupérer toutes les longueurs d'onde des éléments et d'en générer de nouvelles complètement aléatoirement, nous avons décidé de définir à l'avance sur quelles longueurs d'onde travailler.
Concrètement, on "résume" la donnée afin de réduire les informations parasites.

Reprenons notre fonction *generate_spectrums* et ajoutons-y un "pas" permettant de déterminer les longueurs d'ondes à étudier :

In [ ]:
def new_generate_spectrums(base_elements, n, step = 0.5):
    # ne peut pas être construit avec moins d' 1 element
    nb_elems = len(base_elements)

    if (nb_elems < min_elems_used):
        print("not enough base elements provided")
        pass

    # création du df labels
    labels = pd.DataFrame(data = np.zeros((n, nb_elems)), columns=[elem.Name for elem in base_elements])

    # génération des spectres
    list_df = []
    
    for i in range(n):
        # création du squelette + pré-remplissage avec du bruit
        array_wavelength = np.arange(0,max_wl_noise,step)
        array_relint = np.random.randint(min_relint_noise,max_relint_noise, array_wavelength.shape)
        df = pd.DataFrame(data ={"wavelength": array_wavelength , "relint": array_relint})

        # selection des éléments a rajouter
        rand_elems = rand.choices(base_elements, k=rand.randrange(min_elems_used, nb_elems if nb_elems < max_elems_used else max_elems_used))

        # loren ipsum
        for elem in rand_elems:
            # stockage des labels
            labels.at[i, elem.Name] = 1

            # loren ipsum
            elem["wavelength"] = elem["wavelength"].apply(lambda x: myround(x, step))

            # ajout au df
            df = pd.concat([df, elem], ignore_index=True)
        
        # traitement des doublons de wavelength
        df = df.sort_values("relint", ascending=True).drop_duplicates(["wavelength"], keep="last").sort_values("wavelength", ascending=True).reset_index(drop=True)
        list_df.append(df)

    return list_df, labels


def myround(n, base):
    return base * round(n/base)

Un pas de 0,5 nous permet de générer des longueurs d'ondes allant de 0 à 1100 séparées de 0,5 unités. On intègre nos éléments de manière à ce que la valeur la plus haute soit conservée à chaque pas.
Comme avant, on retourne notre liste de dataframe ainsi que nos labels.

Maintenant que nos spectres contiennent tous le même nombre de valeurs, la suite est beaucoup plus simple :

In [ ]:
def reshape(df_list):
    '''
    stocke la donnée en np array 3D
    '''
    np_array = np.array(list(map(lambda x: x.to_numpy(), df_list)))
    print(np_array.shape)

    return np_array

def pca_2D(np_array, n):
    '''
    np_array : une array numpy de dimension 3
    n : le nombre de colonnes finales
    '''
    # flatten
    data_2d = np.array([features_2d.flatten() for features_2d in np_array])
    # min max scaler
    data_scaled = MinMaxScaler().fit(data_2d).transform(data_2d)
    # PCA
    data_pca = PCA(n_components=n).fit_transform(data_scaled)

    print("original shape:   ", data_2d.shape)
    print("transformed shape:", data_pca.shape)

    return data_pca

On passe de plusieurs heures de génération à quelques secondes, avec une donnée dont le sens est beaucoup plus clair.

Plus qu'à nourrir nos modèles :

Avec 2000 spectres, on obtient un résultat de 21% de précision ! La barre des 20% est passée, mais est-ce que l'ont peut aller encore plus loin ?

En réduisant le nombre d'éléments à trouver à 10, on passe à 88% de précision. Avec plus de données d'entraînement, le modèle serait donc capable de déterminer avec précision quels éléments composent chimiquement une étoile à la simple vue de son spectre d'absorption.